<a href="https://colab.research.google.com/github/Re4lmarc/Machine-Learning-Project-Word-Normalization-Hukum-/blob/master/Normalisasi%20Kata%20Bahasa%20Indonesia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mounting Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My\ Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive


## Part Drive Refly


In [2]:
cd Project ML/

/content/drive/My Drive/Project ML


## Part drive Adrian

In [0]:
%cd Colab\ Notebooks

[Errno 2] No such file or directory: 'Colab Notebooks'
/content/drive/My Drive/Project ML


In [0]:
%cd data

[Errno 2] No such file or directory: 'data'
/content/drive/My Drive/Colab Notebooks


# Part Utama

## Instalasi yang dibutuhkan

In [80]:
pip install --upgrade gensim

Requirement already up-to-date: gensim in /usr/local/lib/python3.6/dist-packages (3.8.3)


## Semua Library

In [3]:
# Importing Required Libraries
import pandas as pd
import sqlite3 as sql
import re
import nltk
import glob
import numpy as np
import string
import logging as log
import os
import csv
import pickle
from sklearn import preprocessing
from gensim.models import FastText
from gensim import utils
from gensim.test.utils import datapath
from gensim.utils import tokenize
from string import punctuation
from nltk.tokenize import word_tokenize, RegexpTokenizer
from sqlite3 import Error
from tqdm.autonotebook import tqdm
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report, confusion_matrix

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


## Connect Database

In [0]:
conn = sql.connect("/content/drive/My Drive/Project ML/DBnews.db")
dataset = pd.read_sql_query("SELECT * FROM t_comments", conn)

In [0]:
con2 = sql.connect("/content/drive/My Drive/Project ML/korpus.db")
df_korpus_kata = pd.read_sql_query("SELECT * from korpus_kata", con2)
df_korpus_kalimat = pd.read_sql_query("SELECT * from korpus_kalimat", con2)

In [0]:
dataset['comment'].value_counts()

#2019GantiPresiden                                                                              140
Alhamdulillah                                                                                   130
#2019gantipresiden                                                                              108
%u2063#2019PrabowoPresiden                                                                       84
mantap                                                                                           61
                                                                                               ... 
Pejabat negara jangan memberi contoh tidak baik kepada rakyat dengan tidak menghormati hukum      1
Ijtimak ulama, tapi kok yg hadir banyak orang politik dan bicara capres....?�                     1
Selama anggarannya ada. Zaman now apa masih ada ya anggaran THR buat ormas2 seperti ini ?         1
kita bersuara disini percuma kl ya? g ngaruh jg..g bkl jg dirubah aturannya                       1


## Semua Proses Editing

In [0]:
list_vocab_kalimat = list(df_korpus_kalimat['kalimat'])

In [0]:
for idx_list in range(len(list_vocab_kalimat)):
  list_vocab_kalimat[idx_list] = re.sub('<s>', '', list_vocab_kalimat[idx_list]) 
  list_vocab_kalimat[idx_list] = re.sub('</s>', '', list_vocab_kalimat[idx_list]) 

In [0]:
list_vocab_kalimat[0] = list_vocab_kalimat[0][1:len(list_vocab_kalimat[0])]
list_vocab_kalimat[0]

'penjelasan umum'

In [0]:
kamus_kata_banyak =  open('/content/drive/My Drive/Project ML/ind-id_web_2017_1M/ind-id_web_2017_1M-sentences.txt', encoding='utf-8')
semua_line = kamus_kata_banyak.readlines()

In [0]:
for i in range(len(semua_line)):
  semua_line[i] = a = re.split('\n|\t',semua_line[i])
  semua_line[i] = semua_line[i][1:len(semua_line[i])-1]
  semua_line[i] = semua_line[i][0]

In [0]:
all_comments = list(dataset['comment'])

In [0]:
semua_kalimat_fasttext = semua_line + all_comments + list_vocab_kalimat 

In [0]:
numbers = re.compile(r'[0-9]')
def number(teks):
    teks = [numbers.sub("", word) for word in teks]
    teks = [word for word in teks if len(word) != 0]
    return teks

cleaning = RegexpTokenizer(r'\w+')
def clean(kalimat):
    kata_cleaning = cleaning.tokenize(kalimat)
    return kata_cleaning

def tokenize(kalimat):
    kata_tokenize = word_tokenize(kalimat)
    return kata_tokenize

def case_fold(kalimat):
    kata_lower = []
    for item in kalimat:
        kata_lower.append(str(item).lower())
    return kata_lower

In [0]:
all_filter_fasttext = []
for idx_fasttext in range(len(semua_kalimat_fasttext)):
    token_comments = clean(semua_kalimat_fasttext[idx_fasttext])
    token_comments = number(token_comments)
    token_comments = case_fold(token_comments)
    all_filter_fasttext.append(token_comments)

In [0]:
all_filter_comments = []
for idx_comments in range(len(all_comments)):
    token_comments = clean(all_comments[idx_comments])
    token_comments = number(token_comments)
    token_comments = case_fold(token_comments)
    all_filter_comments.append(token_comments)

## Filter Kata Non KBBI Fase 1

### Filter Kata Baku dengan predefined Corpus

In [0]:
df_korpus_kata = df_korpus_kata.dropna(subset=['kata'], axis=0)
list_vocab_kata = list(df_korpus_kata['kata'])
list_vocab_kalimat = list(df_korpus_kalimat['kalimat'])
list_tag_kata = list(df_korpus_kata['tag'])

In [0]:
not_kbbi = []

dictionary_kata = {}

for kalimat in all_filter_comments:
 for kata in kalimat:
    if kata in list_vocab_kata:
      dictionary_kata[kata] = kata
    else:
      not_kbbi.append(kata)

### Save Kata Baku dan tidak baku yang terfilter 

In [0]:
df_dictionary_kata.columns = ['Kata Sebelum', 'Kata KBBI'] 
df_dictionary_kata
df_dictionary_kata.to_csv('Map Kata KBBI.csv',index=False)

In [0]:
for idx_list in range(len(list_vocab_kalimat)):
  list_vocab_kalimat[idx_list] = re.sub('<s>', '', list_vocab_kalimat[idx_list]) 
  list_vocab_kalimat[idx_list] = re.sub('</s>', '', list_vocab_kalimat[idx_list]) 

In [0]:
kata_non_kbbi = pd.Series(not_kbbi)
kata_non_kbbi.head()

0    wkwkwk
1       dpr
2    mantab
3        br
4        kl
dtype: object

### Load Kata KBBI dan Non KBBI

In [4]:
kata_non_kbbi = pd.read_csv('Map Kata Non KBBI.csv', index_col=[0])
kata_kbbi_pertama = pd.read_excel('Map Kata KBBI2.xlsx')

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [0]:
new_dictionary = dictionary_kata.copy()

for idx in range(len(new_dictionary))

In [0]:
kata_non_kbbi.columns = ['kata tidak baku']
list_kata_non_kbbi = list(kata_non_kbbi['kata tidak baku']) 
print(len(list_kata_non_kbbi))

1967934


## FastText

In [0]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### Kamus Kata benar + dataset

In [0]:
tok_new[0:10]

In [0]:
model = FastText()

#build the vocabulary
model.build_vocab(sentences=tok_new)

# train the model
model.train(sentences=tok_new, epochs=model.epochs, total_examples=model.corpus_count, total_word=model.corpus_total_words)

print(model)

FastText(vocab=21655, size=100, alpha=0.025)


In [0]:
'makin' in model.wv.vocab

True

In [0]:
model.wv.similarity('makan', 'mkn')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


-0.40324718

In [0]:
model.wv.most_similar('makan')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('makanya', 0.9030953645706177),
 ('makanlah', 0.8966188430786133),
 ('makannya', 0.884308934211731),
 ('dimakan', 0.8494226932525635),
 ('termakan', 0.8285702466964722),
 ('makanuntuk', 0.8195314407348633),
 ('makam', 0.8078377842903137),
 ('perlemakan', 0.8075428009033203),
 ('disamakan', 0.807175874710083),
 ('diutamakan', 0.8063433170318604)]

In [0]:
model_fasttext.wv.similarity('siapa', 'siape')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.75340754

### Nama File

In [0]:
filename = 'fasttext_model_fixed.sav'
filename2 = 'fasttext_model_fixed_50.sav'
filename3 = 'fasttext_model_comment.sav'

#### Training & Saving Fasttext Model

In [0]:
model_fasttext = FastText(window=3, min_count=1)
model_fasttext.build_vocab(sentences=all_filter_comments)
total_examples = model_fasttext.corpus_count
model_fasttext.train(sentences=all_filter_comments, total_examples=total_examples, epochs=5)

In [0]:
pickle.dump(model_fasttext, open(filename3, 'wb'))

#### Melakukan Load Model Fasttext yang sudah di Train

In [0]:
# Model Fasttext untuk Kata Gabungan semua Corpus
fasttext_model_50 = pickle.load(open(filename2, 'rb'))

# Model Fasttext Untuk Kata Pada Comment
fasttext_model_comment = pickle.load(open(filename3, 'rb'))

In [0]:
print(fasttext_model_50)
print(fasttext_model_comment)

FastText(vocab=412721, size=50, alpha=0.025)
FastText(vocab=172154, size=100, alpha=0.025)


## Filter Kata Non KBBI Fase 2

In [0]:
df_datapairing = pd.read_csv('https://raw.githubusercontent.com/nasalsabila/kamus-alay/master/colloquial-indonesian-lexicon.csv')

In [0]:
slang_samadengan = df_datapairing['slang'] == 'mkn'
list(df_datapairing['formal'][slang_samadengan].head(1))[0]

'makin'

In [0]:
kata_non_kbbi_dua = []
kata_non_formal = []
kata_formal = []
category_kata = []
vector_sebelum = []
vector_setelah = []

for idx in range(len(list_kata_non_kbbi)):
  slang_samadengan = df_datapairing['slang'] == list_kata_non_kbbi[idx]
  if not df_datapairing[slang_samadengan].empty:
    kata_non_formal.append(list_kata_non_kbbi[idx]) 
    kata_formal.append(list(df_datapairing['formal'][slang_samadengan].head(1))[0])
    category_kata.append(list(df_datapairing['category1'][slang_samadengan].head(1))[0])
    vector_sebelum.append(round(float(fasttext_model.wv[list_kata_non_kbbi[idx]]), 7))
    vector_setelah.append(round(float(fasttext_model.wv[df_datapairing['formal'][slang_samadengan].head(1)]), 7))
  else:
    kata_non_kbbi_dua.append(list_kata_non_kbbi[idx])

In [0]:
len(kata_formal)

df = pd.DataFrame(list(zip(kata_non_formal, kata_formal ,category_kata, vector_sebelum, vector_setelah)), columns =['Kata awal', 'Kata KBBI', 'Kategori', 'Vector Sebelum', 'Vector Setelah']) 

In [0]:
slang_samadengan = df_datapairing['slang'] == 'br'
df_datapairing['formal'][slang_samadengan].head()

True


### Fix Filter 2 Kesalahaan Input

In [0]:
df_kata_kbbi_2_fixed = pd.DataFrame({'Kata Awal' : list_kata_awal, 'Kata KBBI' : list_kata_kbbi, 'Kategori': list_kategori})
df_kata_kbbi_2_fixed.tail()

,Kata Awal,Kata KBBI,Kategori
674424,ngadain,mengadakan,afiksasi
674425,ngadain,mengadakan,afiksasi
674426,nulis,menulis,afiksasi
674427,sebenernya,sebenarnya,modifikasi vokal
674428,bener,benar,modifikasi vokal


In [0]:
df_kata_kbbi_2_fixed.to_csv('Map Kata KBBI 2_fixed.csv',index=False)

### Load Hasil Filter 2

In [0]:
df_kata_kbbi_2_new = pd.read_csv('Map Kata KBBI 2_fixed.csv')

## Filter kata Non KBBI fase 3

In [0]:
kata_non_kbbi = pd.Series(kata_non_kbbi_dua)
kata_non_kbbi.to_csv('Map Kata Non KBBI 2.csv',index=False)

### LCS

In [0]:
def lcs(X, Y, m, n): 
  
    if m == 0 or n == 0: 
       return 0; 
    elif X[m-1] == Y[n-1]: 
       return 1 + lcs(X, Y, m-1, n-1); 
    else: 
       return max(lcs(X, Y, m, n-1), lcs(X, Y, m-1, n)); 

In [0]:
# Melakukan clearning data dari Null Value dan Emotikon
df_kata_non_kbbi = pd.read_csv('Map Kata Non KBBI 2.csv')
df_kata_non_kbbi = df_kata_non_kbbi.dropna()
df_kata_non_kbbi = df_kata_non_kbbi[df_kata_non_kbbi['0'] != 'udd']

In [0]:
df_kata_non_kbbi.columns = ['Kata Sebelum']
df_kata_non_kbbi['Kata KBBI'] = 'none'

In [0]:
tuple_kata_jokowi = fasttext_model_comment.wv.most_similar('jokowi', topn=550)
first_tuple_jokowi = [a_tuple[0] for a_tuple in tuple_kata_jokowi if 'jokowi' in a_tuple[0] or 'joko' in a_tuple[0] or 'owi' in a_tuple[0] or 'jkw' in a_tuple[0]]

In [0]:
tuple_kata_ahok = fasttext_model_comment.wv.most_similar('ahok', topn=550)
first_tuple_ahok = [a_tuple[0] for a_tuple in tuple_kata_ahok if lcs(a_tuple[0], 'ahok', len(a_tuple[0]), 4) >= 3 and a_tuple[0] not in first_tuple_jokowi]

In [0]:
tuple_kata_prabowo = fasttext_model_comment.wv.most_similar('prabowo', topn=550)
first_tuple_prabowo = [a_tuple[0] for a_tuple in tuple_kata_prabowo if lcs(a_tuple[0], 'prabowo', len(a_tuple[0]), len('prabowo')) >= 4 and a_tuple[0] not in first_tuple_jokowi and a_tuple[0] not in first_tuple_ahok]

In [0]:
tuple_kata_nazar = fasttext_model_comment.wv.most_similar('nazar', topn=310)
first_tuple_nazar = [a_tuple[0] for a_tuple in tuple_kata_nazar if lcs(a_tuple[0], 'nazar', len(a_tuple[0]), len('nazar')) >= 4 and a_tuple[0] not in first_tuple_jokowi and a_tuple[0] not in first_tuple_ahok not in first_tuple_prabowo]

In [0]:
tuple_kata_djarot = fasttext_model_comment.wv.most_similar('djarot', topn=100)
first_tuple_djarot = [a_tuple[0] for a_tuple in tuple_kata_djarot if lcs(a_tuple[0], 'djarot', len(a_tuple[0]), len('djarot')) >= 4 and a_tuple[0] not in first_tuple_jokowi and a_tuple[0] not in first_tuple_ahok not in first_tuple_prabowo not in first_tuple_nazar]

In [0]:
join_tuple_check = first_tuple_jokowi + first_tuple_ahok + first_tuple_prabowo + first_tuple_djarot + first_tuple_nazar

In [0]:
tuple_kata_haha = fasttext_model_comment.wv.most_similar('wkwk', topn=100)
first_tuple_haha = [a_tuple[0] for a_tuple in tuple_kata_haha if lcs(a_tuple[0], 'wkwk', len(a_tuple[0]), len('wkwk')) >= 2]

In [0]:
df_kata_non_kbbi["Kata KBBI"][df_kata_non_kbbi["Kata Sebelum"].isin(first_tuple_nazar+["nazar"])]

In [0]:
tuple_kata_tuh = fasttext_model_comment.wv.most_similar('tuh', topn=100)
first_tuple_tuh = [a_tuple[0] for a_tuple in tuple_kata_tuh if lcs(a_tuple[0], 'tuh', len(a_tuple[0]), len('tuh')) >= 2]

In [0]:
first_tuple_djarot = first_tuple_djarot + ['djarot']
first_tuple_ahok = first_tuple_ahok + ['ahok']
first_tuple_jokowi = first_tuple_jokowi + ['jokowi']
first_tuple_nazar = first_tuple_nazar + ['nazar']
first_tuple_prabowo = first_tuple_prabowo + ['prabowo']
first_tuple_haha = first_tuple_haha + ['wkwk']
first_tuple_tuh = first_tuple_tuh + ['tuh']

all_join = []
all_join.append(first_tuple_djarot)
all_join.append(first_tuple_ahok)
all_join.append(first_tuple_jokowi)
all_join.append(first_tuple_nazar)
# all_join.append(first_tuple_haha)
all_join.append(first_tuple_prabowo)

In [0]:
for i in range(len(all_join)):
  df_kata_non_kbbi["Kata KBBI"][df_kata_non_kbbi["Kata Sebelum"].isin(all_join[i])] = all_join[i][len(all_join[i])-1]

In [0]:
cari = df_kata_non_kbbi["Kata Sebelum"] == "wowo"
df_kata_non_kbbi[cari]

,Kata Sebelum,Kata KBBI
125,wowo,none
136,wowo,none
159,wowo,none
1490,wowo,none
1601,wowo,none
...,...,...
1293220,wowo,none
1293252,wowo,none
1293293,wowo,none
1293351,wowo,none


In [0]:
df_kata_non_kbbi["Kata KBBI"][df_kata_non_kbbi["Kata Sebelum"].isin(first_tuple_haha)] = 'haha'
df_kata_non_kbbi["Kata KBBI"][df_kata_non_kbbi["Kata Sebelum"].isin(first_tuple_tuh)] = 'itu'

In [0]:
df_kata_non_kbbi.to_csv('Map Kata Non KBBI 3.csv',index=False)

### Load Hasil filter 3

In [0]:
df_kata_non_kbbi = pd.read_csv('Map Kata Non KBBI 3.csv')

# Gabung Semua DataFrame

In [0]:
df_kata_kbbi_2_new.columns = ["Kata Sebelum", "Kata KBBI", "Kategori"]

In [0]:
frames = [kata_kbbi_pertama, df_kata_kbbi_2_new, df_kata_non_kbbi]
df_all = pd.concat(frames, ignore_index=True) 

In [0]:
df_all = df_all.replace(np.nan, 'Tidak Tahu', regex=True)
df_all = df_all.replace('none', 'Tidak Tahu', regex=True)

In [10]:
df_all['Kata KBBI'].value_counts().head()

Tidak Tahu    1143024
enggak          76101
jokowi          43665
prabowo         28928
sudah           22165
Name: Kata KBBI, dtype: int64

In [0]:
list_sebelum_all = list(df_all['Kata Sebelum'])
list_kbbi_all = list(df_all['Kata KBBI'])

In [0]:
df_category_sebelum = df_all["Kata Sebelum"].value_counts().to_frame()
df_category_sebelum = df_category_sebelum.reset_index()
list_df_category = list(df_category_sebelum["index"])
df_category_kbbi = df_all["Kata KBBI"].value_counts().to_frame()
df_category_kbbi = df_category_kbbi.reset_index()
list_df_category_kbbi = list(df_category_kbbi["index"])

In [13]:
df_category_sebelum.columns = ["Kata Sebelum", "Counts Sebelum"]
df_category_sebelum.head()

df_category_kbbi.columns = ["Kata KBBI", "Counts KBBI"]
df_category_kbbi.head()

,Kata KBBI,Counts KBBI
0,Tidak Tahu,1143024
1,enggak,76101
2,jokowi,43665
3,prabowo,28928
4,sudah,22165


In [14]:
le_kata_sebelum = preprocessing.LabelEncoder()
le_kata_kbbi = preprocessing.LabelEncoder()
le_kata_sebelum.fit(list_df_category)
le_kata_kbbi.fit(list_df_category_kbbi)

LabelEncoder()

### Transform Kamus (Not used now)

In [0]:
# list_transformed_sebelum_vocab = le_kata_sebelum.transform(list_df_category)
# list_transformed_kbbi_vocab = le_kata_kbbi.transform(list_df_category_kbbi)

In [0]:
split_length = int(len(list_sebelum_all) / 2)

### Transform Whole Dataset

In [0]:
list_transformed_sebelum = list(le_kata_sebelum.transform(list_sebelum_all[:split_length]))
list_transformed_sebelum_2 = list(le_kata_sebelum.transform(list_sebelum_all[split_length:]))
list_transformed_kbbi = list(le_kata_kbbi.transform(list_kbbi_all[:split_length]))
list_transformed_kbbi_2 = list(le_kata_kbbi.transform(list_kbbi_all[split_length:]))

In [20]:
list_transformed_kbbi[0: 100]

[5994,
 18165,
 14518,
 18260,
 8383,
 8134,
 1260,
 9822,
 4730,
 7288,
 3460,
 21955,
 18147,
 3411,
 20131,
 21421,
 10904,
 2330,
 10776,
 17717,
 2659,
 6568,
 14201,
 15930,
 10583,
 3486,
 1011,
 12483,
 18145,
 19488,
 8071,
 7555,
 66,
 2932,
 3652,
 8410,
 810,
 10182,
 16416,
 20218,
 10384,
 1089,
 204,
 10948,
 7131,
 18735,
 21377,
 16769,
 7590,
 11257,
 7825,
 8131,
 11514,
 10345,
 3673,
 20786,
 11064,
 7895,
 6758,
 7531,
 10445,
 7145,
 7530,
 19391,
 12848,
 9299,
 14691,
 8089,
 11254,
 1683,
 14419,
 16366,
 3072,
 18022,
 2287,
 2684,
 200,
 1442,
 19553,
 13999,
 3786,
 15592,
 8308,
 15462,
 9072,
 12177,
 5032,
 11259,
 931,
 19518,
 19153,
 13864,
 4956,
 18408,
 8060,
 11012,
 3955,
 17778,
 18950,
 17942]

In [0]:
list_transformed_sebelum_join = list_transformed_sebelum+list_transformed_sebelum_2 
list_transformed_kbbi_join = list_transformed_kbbi+list_transformed_kbbi_2

In [0]:
dictionary_sebelum = dict(zip(list_df_category, list_transformed_sebelum))
dictionary_kbbi = dict(zip(list_df_category_kbbi, list_transformed_kbbi))

## Coba Coba

In [0]:
df_tes = pd.DataFrame({"Kata Sebelum" : list_sebelum_all, "Kata KBBI" : list_kbbi_all, "Encoded Kata Sebelum" : list_transformed_sebelum_join, "Encoded Kata KBBI" : list_transformed_kbbi_join})

In [0]:
df_tes.to_csv('Map Kata Setelah Encode.csv',index=False)

### Load CSV Setelah Encode

In [0]:
df_tes = pd.read_csv('Map Kata Setelah Encode.csv')

### Mengecilkan Train data

In [0]:
df_train = df_tes[['Kata Sebelum', 'Kata KBBI' ,'Encoded Kata Sebelum', 'Encoded Kata KBBI']].drop_duplicates()
# num_class = df_train['Encoded Kata KBBI'].drop_duplicates()
# num_class = len(num_class)
# print(num_class)

df_train.reset_index(drop=True, inplace=True)

### Menginisiasi Model Decision Tree Classifier

In [0]:
from sklearn.tree import DecisionTreeClassifier

X_train = df_train['Encoded Kata Sebelum']
y_train = df_train['Encoded Kata KBBI']
# model_logreg = LogisticRegression()
clf = DecisionTreeClassifier(criterion='gini',  min_samples_leaf=1)

### Menentukan Train Data

In [0]:
X_train = X_train[0:10000]
y_train = y_train[0:10000]

X_train = np.append(X_train, 64422)
y_train = np.append(y_train, 7864)
X_train = np.append(X_train, 127961)
y_train = np.append(y_train, 16464)
X_train = np.append(X_train, 17730)
y_train = np.append(y_train, 2696)

In [30]:
X_train.shape

(10003, 1)

In [0]:
X_train = X_train[ :, np.newaxis]

### Melakukan Training

In [0]:
clf.fit(X_train, y_train)

In [0]:
pickle.dump(clf, open("Model_Decision_Tree_Classifier.sav", 'wb'))

In [0]:
clf = pickle.load(open("Model_Decision_Tree_Classifier.sav", 'rb'))

### Melakukan Prediksi dan mendapatkan Akurasi

In [0]:
y_pred_train = clf.predict(X_train)

In [33]:
y_pred_train[1:100]

array([18165, 14518, 18260,  8383,  8134,  1260,  9822,  4730,  7288,
        3460, 21955, 18147,  3411, 20131, 21421, 10904,  2330, 10776,
       17717,  2659,  6568, 14201, 15930, 10583,  3486,  1011, 12483,
       18145, 19488,  8071,  7555,    66,  2932,  3652,  8410,   810,
       10182, 16416, 20218, 10384,  1089,   204, 10948,  7131, 18735,
       21377, 16769,  7590, 11257,  7825,  8131, 11514, 10345,  3673,
       20786, 11064,  7895,  6758,  7531, 10445,  7145,  7530, 19391,
       12848,  9299, 14691,  8089, 11254,  1683, 14419, 16366,  3072,
       18022,  2287,  2684,   200,  1442, 19553, 13999,  3786, 15592,
        8308, 15462,  9072, 12177,  5032, 11259,   931, 19518, 19153,
       13864,  4956, 18408,  8060, 11012,  3955, 17778, 18950, 17942])

In [34]:
print("Tingkat Accuracy :",  accuracy_score(y_train, y_pred_train))
# print(confusion_matrix(y_train, y_pred_train))

Tingkat Accuracy : 0.9999000299910027


In [35]:
print(classification_report(y_train, y_pred_train))

              precision    recall  f1-score   support

           0       0.50      1.00      0.67         1
           1       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         1
           6       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         1
           8       1.00      1.00      1.00         1
          10       1.00      1.00      1.00         1
          11       1.00      1.00      1.00         1
          14       1.00      1.00      1.00         1
          15       1.00      1.00      1.00         1
          16       1.00      1.00      1.00         1
          17       1.00      1.00      1.00         1
          21       1.00      1.00      1.00         1
          23       1.00      1.00      1.00         1
          27       1.00      1.00      1.00         1
          34       1.00      1.00      1.00         1
          36       1.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Test ke String Manual

In [39]:
string_test = ['jkw', 'yg', 'menang', 'bkn', 'pranowo']
kalimat_awal = le_kata_sebelum.transform(string_test)
print("String Awal:", string_test)
print("Nilai Encoding Awal :", kalimat_awal)
kalimat_awal = kalimat_awal[ :, np.newaxis]

# Prediksi
kalimat_prediksi = clf.predict(kalimat_awal)
print()

print("Nilai Encoding Prediksi", kalimat_prediksi)
print("String Setelah Prediksi:", le_kata_kbbi.inverse_transform(kalimat_prediksi))

String Awal: ['jkw', 'yg', 'menang', 'bkn', 'pranowo']
Nilai Encoding Awal : [ 64422 170948  93087  17730 127961]

Nilai Encoding Prediksi [ 7864 21955 12020  2696 16464]
String Setelah Prediksi: ['jokowi' 'yang' 'menang' 'bukan' 'prabowo']
